# Data Control
The Datasheet has many different problems
- Not having data for a specific criteria
- Data is not formatted correctly

In [ ]:
import os
import time
import json
import ast
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
import statistics
import numpy as np
import random
from collections import Counter

In [ ]:
os.chdir("/Users/kunal/Documents/AAPersonalAIPROJECT/HATCH")
for i in os.listdir("."):
    print(i)

In [ ]:
ethnicities = []
f = open("ethnicities.txt", "r", encoding="utf8")
for x in f:
    ethnicities.append(x[:-1].lower().strip())

In [ ]:
arryStrings = []
f = open("DomoArigatoData.txt", "r")
for x in f:
    arryStrings.append(x)

In [ ]:
for i in arryStrings[1:]:
    if not (i[0:5] == "FALSE" or i[0:4] == "TRUE"):
        print(i)

In [ ]:
def gene(line, value):
    ct = 0
    while ct < 10:
        if line[ct] == "\t" and line[ct+1] == "\t":
            val= False
            return("No Gene")
            break
        ct+=1
    ct2=0
    string = ""
    while ct2 < 15:
        if line[ct2] == "\t":
            for i in line[ct2+1:]:
                string+=i
                if i =="\t":
                    return(string.replace("\t",""))
                    break
        ct2+=1

In [ ]:
def calc(line):
    ct = 0
    string1 = ""
    while ct < 30:
        if line[ct] == "\t" and line[ct+1] == "\t":
            #print("No Gene")
            #print(line[ct+2:])
            for i in line[ct+2:]:
                string1+=i
                if i =="\t":
                    #print(string.replace("\t",""))
                    return "No Gene", string1.replace("\t","").strip()
                    break
            break
        ct+=1
    ct2=0
    string = ""
    stringNew = ""
    while ct2 < 15:
        if line[ct2] == "\t":
            ctNew = 0
            for i in line[ct2+1:]:
                string+=i
                if i =="\t":
                    #print(string.replace("\t",""))
                    break
                ctNew+=1
            #print(line[ct2+ctNew+2:])
            for j in line[ct2+ctNew+2:]:
                #print(j)
                stringNew += j
                if j =="\t":
                    #print(stringNew.replace("\t",""))
                    return string.replace("\t","").strip(), stringNew.replace("\t","").strip()
                    break
        ct2+=1

In [ ]:
%%time
convertArray = []
for line in arryStrings[1:]:
    tempArray = []
    if line[0:5].strip().upper() == "FALSE":
        tempArray.append(["Pathogenic", "False"])  
    elif line[0:4].strip().upper() == "TRUE":
        tempArray.append(["Pathogenic", "True"])  
    else:
        raise Exception("Invalid first Value\"{}\"".format(str(line)))
    geneVal, history_classVal = calc(line)
    tempArray.append(["Gene",geneVal])
    tempArray.append(["History_class", history_classVal])
    if not line.find("\"") == -1:
        runOther = line[line.find("\"")+1:len(line)-1]
    for i in runOther.split(";"):
        i = i[:-5] if i.endswith("\t\t\t\t") else i
        details = i.strip().split(":")
        try:
            details[1] = details[1][:-1] if details[1].endswith("\"") else details[1]
            details[1] = json.loads(details[1].replace("\"\"","\""))
        except:
            pass
        tempArray.append(details)
    convertArray.append(tempArray)

In [ ]:
ctRemoved = 0
ctPointRemovePointer = 0
pointsRemoved = []
possible = ['Pathogenic','Gene','History_class','consent_approval','ethnicity','cancer_dx','cancer_dx_type','relationships',
            'cancer_dx_age','known_brca','known_cancer','rel_relation','rel_cancer','rel_age', '_method', 'other_cancer']
for singleEntity in convertArray:
    for criteria in singleEntity:
        if not criteria[0] in possible:
            pointsRemoved.append(["Title Wrong", ctPointRemovePointer])
            print(criteria)
            #print(singleEntity, end="\n\n")
            #convertArray.remove(singleEntity)
            ctRemoved +=1
    ctPointRemovePointer+=1
print("{} different entities due to bad data format".format(ctRemoved))

In [ ]:
for i in pointsRemoved:
    del convertArray[i[1]]
pointsRemoved = []

# Visulization of Pathogenic

In [ ]:
total = len(convertArray)
cttrue = 0
ctfalse = 0
for i in convertArray:
    cttrue += 1 if i[0][1] == 'True' else 0
    ctfalse += 1 if i[0][1] == 'False' else 0
print(round(cttrue/total * 100,2),round(ctfalse/total * 100,2))
plt.title("Pathogenic")
plt.pie([cttrue, ctfalse], labels = ["True " + str(round(cttrue/total * 100,3)), 
                                     "False " + str(round(ctfalse/total * 100,3))])
plt.show()

# Gene

In [ ]:
possibleGene = ["NCCN", "HBOC", "MEN2", "PTEN", "MEN1", "ACMG", "FAMMM"]

In [ ]:
ctGene = 0
for i in possibleGene: 
    if i != 'No Gene': ctGene+=1
lstOfGene = []
for i in convertArray:
    #if i[1][1] != "No Gene": print(i[1][1]) 
    lstOfGene.append(i[1][1])
#print("-"*20)
print("{} different Genes with total {} gene locations".format(len(set(lstOfGene)), ctGene))

# Delete bad data

### ```consent_approval```

There are only 2 possible options for this criteria, `yes` and `no` so adjusting all of them to that

In [ ]:
for i in convertArray:
    for j in i:
        if j[0] == "consent_approval":
            j[1] = j[1].lower().strip()
            #Printing all possible wrong points
            if j[1] != "yes" and j[1] != "no":
                print(j[1])

### ```ethnicity```

In [ ]:
from spellchecker import SpellChecker
spell = SpellChecker()

In [ ]:
def ethnicityChange(listofVals):
    listofVals = listofVals.lower().replace("/",",").replace("caucasion","caucasian").replace("-",",").replace("and",",").replace("&",",").replace("english", "").replace("white", "caucasian").strip().split(",")
    #print(listofVals)
    updated = listofVals
    ct = 0
    for i in listofVals:
        if "not" in i or "non" in i:
            try:
                updated.remove(listofVals[ct+1])
            except:
                pass
        ct+=1

    lst = [value for value in updated if value != ""]
    #print(lst)
    ct = 0
    for j in lst:
        if j != spell.correction(j):
            lst[ct] = spell.correction(j)
            #print(j, spell.correction(j))
        ct+=1
    #print(lst)
    final = []
    for i in lst:
        if i in ethnicities:
            final.append(i)
        else:
            splitFinalVal = i.split(" ")
            for j in splitFinalVal:
                #print(j)
                if j in ethnicities:
                    final.append(j)
        #else:
            #print("Removed: ", i)
    return final

In [ ]:
for i in convertArray:
    for j in i:
        if j[0] == "ethnicity":
            answers = ethnicityChange(j[1])
            if len(answers) > 2:
                a = random.sample(range(0, len(answers)), 2)
                j[1] = [answers[a[0]],answers[a[1]]]
            elif len(answers) == 2:
                j[1] = answers
            elif len(answers) == 1:
                j[1] = [answers[0],""]
            elif len(answers) == 0:
                j[1] = ["",""]
            else:
                raise Exception("Error Code")

### ```cancer_dx```

only yes or no answer

In [ ]:
for i in convertArray:
    for j in i:
        if j[0] == "cancer_dx":
            j[1] = j[1].lower().strip()

### ```cancer_dx_type```


Going to drop look into later

In [ ]:
for i in convertArray:
    for j in i:
        if j[0] == "cancer_dx_type":
            j[1] = j[1].lower().strip()
            print(j)

### ```relationships```

In [ ]:
ctPointDelete = 0
for i in convertArray:
    for j in i:
        if j[0] == "relationships":
            #j[1] = j[1].lower().strip()
            pointsRemoved.append(["Relationships",ctPointDelete])
            #print(j[1])
    ctPointDelete+=1

In [ ]:
for i in pointsRemoved:
    del convertArray[i[1]]
pointsRemoved = []

### ```cancer_dx_age```
average if multiple

In [ ]:
ctRemovedAge = 0
for i in convertArray:
    for j in i:
        if j[0] == "cancer_dx_age":
            #j[1] = j[1].replace()
            if type(j[1]) != int and j[1] != '':
                r = j[1].strip().replace("and",",").split(",")
                r = [s.strip() for s in r]
                dele = False
                for vla in r:
                    try:
                        vla = int(vla)
                    except:
                        dele = True
                        convertArray.remove(i)
                        ctRemovedAge+=1
                if not dele: j[1] = int(statistics.mean(map(int, r)) )
print("Deleted {}".format(ctRemovedAge))

### ```known_brca```

In [ ]:
for i in convertArray:
    for j in i:
        if j[0] == "known_brca":
            j[1] = j[1].replace("U","unknown").lower().strip()
            #print(j)

### ```rel_relation```

In [ ]:
import string

In [ ]:
CounterList = []
exclude = set(string.punctuation)
for i in convertArray:
    for j in i:
        if j[0] == "rel_relation":
            #print(j)
            newList = []         
            for valr in j[1]:
                value = valr.lower().replace("paternal","").replace("mom","mother").replace("grandpa","grandfather")
                value = value.replace("grandma","grandmother").replace("first","").replace("dad","father").replace("1st","")
                value = value.replace("maternal","").replace("greatgrandmother", "great grandmother")
                value = ''.join(ch for ch in value if ch not in exclude)
                newList.append(value.strip())
            if len(newList) > 1 and "" in newList:
                newList.remove("")
            j[1] = list(set(newList))
            CounterList.extend(list(set(newList)))
            #print(j)

In [ ]:
A = Counter(CounterList)
r=0
lol = 0
removeLst = []
for key, value in A.items():
    if value > 10:
        #print(key)#
        lol+=value
    else:
        r+=1

In [ ]:
possibleRelationship = ["aunt", "brother", "cousin", "daughter", "father", "grandfather", "grandmother", "great aunt", 
                        "great grandfather", "great grandmother", "great uncle", "mother", "niece", "sister", "uncle"]

In [ ]:
for i in convertArray:
    for j in i:
        if j[0] == "rel_relation":
            newList = []
            for valr in j[1]:
                if not valr in possibleRelationship:
                    newList.append("")
                else:
                    newList.append(valr)
            newList = list(set(newList))
            if ("" in newList) and (not "" in newList) and len(newList) != 1:
                newList.remove("")
            j[1] = newList
            print(j)
#possibleRelationship.append("")

### ```rel_cancer```

In [ ]:
%%time
R = []
relCancerPoss = ["bile duct", "bladder", "bone", "brain", "breast", "cervical", "colon", "colorectal", "endometrial", 
                 "esophageal", "kidney", "leukemia", "liver", "lung", "lymphoma", "melanoma", "mesothelioma", "mouth", 
                 "myeloma", "non hodgkins lymphoma", "ovarian", "pancreas", "pancreatic", "prostate", "prostrate", "rectal",
                 "skin", "stomach", "testicular", "throat", "thyroid", "uterine"]
exclude = set(string.punctuation)
for i in convertArray:
    for j in i:
        if j[0] == "rel_cancer":
            tst = []
            #print(j[1])
            for valr in j[1]:
                point = valr.lower().replace("cancer","").replace("prostrate","prostate").replace("unknown","").replace("multiple","").replace("leukemia cml", "leukemia").replace("tumor","").strip()
                point = ''.join(ch for ch in point if ch not in exclude)
                #print(point)
                for i in relCancerPoss:
                    if i in point:
                        point = i  
                if not point in relCancerPoss and point != "":
                    spellChange = spell.correction(point)
                    if point != spellChange:
                        point = spellChange
                tst.append(point)
            if len(tst) > 1 and "" in tst:
                tst.remove("")
            j[1] = list(set(tst))
            print(j)
            R.extend(j[1])
#relCancerPoss.append("")

In [ ]:
for i in convertArray:
    for j in i:
        if j[0] == "rel_cancer":
            print(j)

### ```rel_age```

In [ ]:
exclude = set(string.punctuation)
ctdelRel_AGE = 0
pointersToRemove = 0
for i in convertArray:
    for j in i:
        if j[0] == "rel_age":
            newListAge = []
            for val in j[1]:
                valr = val.replace("unknown", "").replace("?", "").replace("late", "").replace("ish", "").replace("s", "").replace("'", "").replace("appx", "").replace("approx.", "").replace("~", "").replace("Over", "").strip()
                valr = ''.join(ch for ch in valr if ch not in exclude)
                try:
                    valr = valr.lower().strip().replace("n/a","").replace("not sure","").replace("not ure","").replace("non","").replace("unure","").replace("Ukwn","").replace("unk","").replace("unknown","").replace("Unkown","").replace("UK","")
                except:
                    print("Error Check - ", valr)
                    raise Exception("Changing Text")
                if valr != "":
                    try:
                        newListAge.append(int(valr))            
                    except:
                        print("Deleting:", valr)
            if len(newListAge) > 3:
                a,b,c = np.percentile([67, 35, 64, 74, 82, 55], [25, 50, 75])
                j[1] = [int(a),int(b),int(c)]
            elif len(newListAge) == 3:
                j[1] = newListAge
            elif len(newListAge) == 2:
                j[1] = [newListAge[0], newListAge[1], 0] 
            elif len(newListAge) == 1:
                j[1] = [newListAge[0], 0, 0] 
            else:
                print(j)
                j[1] = [0,0,0]
    pointersToRemove+=1
print(ctdelRel_AGE)

### ```_method```

Since this column is esentially useless, I will drop it

In [ ]:
#for i in convertArray:
    #for j in i:
        #if j[0] == "_method":
            #print(j)

### ```other_cancer```

Too few data points

In [ ]:
for i in convertArray:
    for j in i:
        if j[0] == "other_cancer":
            print(j)

# Show Data

In [ ]:
def conv2dLstTo1d(array):
    r = []
    for i in array:
        if type(i) == list:
            for j in i:
                r.append(j)
        else:
            r.append(i)
    return r

In [ ]:
display = []
titles = ["Name","Percentage", "Count", "Types", "Example 5 data"]
lstOfAllData = []
total = 0
for txt in possible:
    tempArrayTypes=[]
    ctVal = 0
    ctR = 0
    for i in convertArray:
        hasValue = False
        for j in i:
            if j[0] == txt:
                hasValue = True
                tempArrayTypes.append(j[1])
                ctR+=1
        if not hasValue: ctVal += 1
    total+=1
    display.append([txt, (ctVal/total)*100, ctR,len(set(conv2dLstTo1d(tempArrayTypes))), 
                    list(Counter(conv2dLstTo1d(tempArrayTypes)).keys())[0:5]])
    lstOfAllData.append([txt, list(Counter(conv2dLstTo1d(tempArrayTypes)).keys())])
    

In [ ]:
df = pd.DataFrame(display, columns = titles)

In [ ]:
df

In [ ]:
for i in pointsRemoved:
    del convertArray[i[1]]

# Encoding

In [ ]:
saveMe = convertArray

In [ ]:
convertArray = saveMe

In [ ]:
requiredTitles = ['Pathogenic','Gene','History_class','consent_approval','ethnicity','cancer_dx','cancer_dx_type',
                  'relationships','cancer_dx_age','known_brca','known_cancer','rel_relation','rel_cancer','rel_age',
                  '_method','other_cancer']
finalData = []
for i in convertArray:
    tempArray = i
    titlesArrayTemp = []
    for j in tempArray:
        titlesArrayTemp.append(j[0])
    for k in requiredTitles:
        if not k in titlesArrayTemp:
            if k == "rel_age":
                tempArray.append([k,[0,0,0]])
            else:
                tempArray.append([k,"-"])
    tempArray.sort()
    finalData.append(tempArray)

In [ ]:
count = 0
for i in finalData:
    if (len(i)) != 16:
        del finalData[count]
        print("Deleted.")
    count+=1

In [ ]:
len(finalData)

In [ ]:
def encodeRelationship(lst):
    string = ""
    for rel in possibleRelationship:
        if rel in lst:
            string+="1"
        else:
            string+="0"
    return string

In [ ]:
def encodeRelCancer(lst):
    string = ""
    for rel in relCancerPoss:
        if rel in lst:
            string+="1"
        else:
            string+="0"
    return string

In [ ]:
finalDataDF = []
for i in finalData:
    tempArray = []
    for j in i:
        if j[0] == "ethnicity":
            tempArray.append(j[1][0])
            tempArray.append(j[1][1])
        elif j[0] == "rel_relation":
            tempArray.append(encodeRelationship(j[1]))
        elif j[0] == "rel_cancer":
            tempArray.append(encodeRelCancer(j[1]))
        elif j[0] == "rel_age":
            try:
                tempArray.append(j[1][0])
                tempArray.append(j[1][1])
                tempArray.append(j[1][2])
            except IndexError:
                print("Error with data defaulting to 0")
                print(j)
                tempArray.append(0)
                tempArray.append(0)
                tempArray.append(0)
        elif j[0] == "cancer_dx_age":
            if j[1] == "":
                tempArray.append(0)
            else:
                tempArray.append(j[1])
        else:
            tempArray.append(j[1])
        print(j)
    print("")
    finalDataDF.append(tempArray)

In [ ]:
dfArray = []
for i in finalDataDF:
    if len(i) == 19:
        dfArray.append(i)

In [ ]:
df = pd.DataFrame(dfArray, columns = ['Gene', 'History_class', 'Pathogenic','_method', 'cancer_dx','cancer_dx_age',
                                      'cancer_dx_type', 'consent_approval','ethnicity_1','ethnicity_2', 'known_brca', 
                                      'known_cancer', 'other_cancer', 'rel_age_1', 'rel_age_2', 'rel_age_3', 'rel_cancer', 
                                      'rel_relation', 'relationships'])

In [ ]:
df

# Lets do this

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import math
import tensorflow as tf
import numpy as np
from numpy import unique
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, TensorBoard

In [ ]:
dataframe = df.drop(['Gene','Pathogenic','_method', 'relationships', 'consent_approval'], axis=1)

In [ ]:
val_dataframe = dataframe.sample(frac=0.2, random_state=1337)
pred_dataframe = dataframe.sample(frac=0.1, random_state=1337)
train_dataframe = dataframe.drop(val_dataframe.index)

In [ ]:
print("Using %d samples for training, %d for validation and %d for predicting" 
      % (len(train_dataframe), len(val_dataframe), len(pred_dataframe)))

def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("History_class")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds


def pred_dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe)))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds



train_ds = dataframe_to_dataset(train_dataframe)
val_ds = dataframe_to_dataset(val_dataframe)
pred_ds = pred_dataframe_to_dataset(pred_dataframe)

for x, y in train_ds.take(1):
    print("Input:", x)
    print("Target:", y)

train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)

In [ ]:
train_ds.take(1)

In [ ]:
{'cancer_dx': <tf.Tensor: shape=(), dtype=string, numpy=b'no'>, 
 'cancer_dx_age': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 
 'cancer_dx_type': <tf.Tensor: shape=(), dtype=string, numpy=b''>, 
 'ethnicity_1': <tf.Tensor: shape=(), dtype=string, numpy=b'caucasian'>, 
 'ethnicity_2': <tf.Tensor: shape=(), dtype=string, numpy=b''>, 
 'known_brca': <tf.Tensor: shape=(), dtype=string, numpy=b'yes'>, 
 'known_cancer': <tf.Tensor: shape=(), dtype=string, numpy=b'yes'>, 
 'other_cancer': <tf.Tensor: shape=(), dtype=string, numpy=b'-'>, 
 'rel_age_1': <tf.Tensor: shape=(), dtype=int64, numpy=60>, 
 'rel_age_2': <tf.Tensor: shape=(), dtype=int64, numpy=70>, 
 'rel_age_3': <tf.Tensor: shape=(), dtype=int64, numpy=50>, 
 'rel_cancer': <tf.Tensor: shape=(), dtype=string, numpy=b'00001000000000000000100000000000'>, 
 'rel_relation': <tf.Tensor: shape=(), dtype=string, numpy=b'100000100000000'>}

In [ ]:
Target: tf.Tensor(b'strong_family', shape=(), dtype=string)


In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers.experimental.preprocessing import CategoryEncoding
from tensorflow.keras.layers.experimental.preprocessing import StringLookup


def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature


def encode_string_categorical_feature(feature, name, dataset):
    # Create a StringLookup layer which will turn strings into integer indices
    index = StringLookup()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the set of possible string values and assign them a fixed integer index
    index.adapt(feature_ds)

    # Turn the string input into integer indices
    encoded_feature = index(feature)

    # Create a CategoryEncoding for our integer indices
    encoder = CategoryEncoding(output_mode="binary")

    # Prepare a dataset of indices
    feature_ds = feature_ds.map(index)

    # Learn the space of possible indices
    encoder.adapt(feature_ds)

    # Apply one-hot encoding to our indices
    encoded_feature = encoder(encoded_feature)
    return encoded_feature


def encode_integer_categorical_feature(feature, name, dataset):
    # Create a CategoryEncoding for our integer indices
    encoder = CategoryEncoding(output_mode="binary")

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the space of possible indices
    encoder.adapt(feature_ds)

    # Apply one-hot encoding to our indices
    encoded_feature = encoder(feature)
    return encoded_feature

In [ ]:
Cancer_dx_Age = keras.Input(shape=(1,), name="cancer_dx_age", dtype="int64")
Relative_Age_1 = keras.Input(shape=(1,), name="rel_age_1", dtype="int64")
Relative_Age_2 = keras.Input(shape=(1,), name="rel_age_2", dtype="int64")
Relative_Age_3 = keras.Input(shape=(1,), name="rel_age_3", dtype="int64")


Relative_Cancer = keras.Input(shape=(1,), name="rel_cancer", dtype="string")
Relative_Relation = keras.Input(shape=(1,), name="rel_relation", dtype="string")
Cancer_Dx = keras.Input(shape=(1,), name="cancer_dx", dtype="string")
Cancer_Dx_Type = keras.Input(shape=(1,), name="cancer_dx_type", dtype="string")
Ethnicity_1 = keras.Input(shape=(1,), name="ethnicity_1", dtype="string")
Ethnicity_2 = keras.Input(shape=(1,), name="ethnicity_2", dtype="string")
Known_BRCA = keras.Input(shape=(1,), name="known_brca", dtype="string")
Known_Cancer = keras.Input(shape=(1,), name="known_cancer", dtype="string")
Other_Cancer = keras.Input(shape=(1,), name="other_cancer", dtype="string")

#Cancer_dx_Age = keras.Input(shape=(1,), name="cancer_dx_age")
#Relative_Age_1 = keras.Input(shape=(1,), name="rel_age_1")
#Relative_Age_2 = keras.Input(shape=(1,), name="rel_age_2")
#Relative_Age_3 = keras.Input(shape=(1,), name="rel_age_3")


all_inputs = [
    Relative_Cancer,
    Relative_Relation,
    Cancer_Dx,
    Cancer_Dx_Type,
    Ethnicity_1,
    Ethnicity_2,
    Known_BRCA,
    Known_Cancer,
    Other_Cancer,

    Cancer_dx_Age,
    Relative_Age_1,
    Relative_Age_2,
    Relative_Age_3]

In [ ]:
# Integer categorical features
#Education_encoded = encode_integer_categorical_feature(Education, "Education", train_ds)


# String categorical features
Relative_Cancer_encoded = encode_string_categorical_feature(Relative_Cancer, "rel_cancer", train_ds)
Relative_Relation_encoded = encode_string_categorical_feature(Relative_Relation, "rel_relation", train_ds)
Cancer_Dx_encoded = encode_string_categorical_feature(Cancer_Dx, "cancer_dx", train_ds)
Cancer_Dx_Type_encoded = encode_string_categorical_feature(Cancer_Dx_Type, "cancer_dx_type", train_ds)
Ethnicity_1_encoded = encode_string_categorical_feature(Ethnicity_1, "ethnicity_1", train_ds)
Ethnicity_2_encoded = encode_string_categorical_feature(Ethnicity_2, "ethnicity_2", train_ds)
Known_BRCA_encoded = encode_string_categorical_feature(Known_BRCA, "known_brca", train_ds)
Known_Cancer_encoded = encode_string_categorical_feature(Known_Cancer, "known_cancer", train_ds)
Other_Cancer_encoded = encode_string_categorical_feature(Other_Cancer, "other_cancer", train_ds)

# Numerical features
Cancer_dx_Age_encoded = encode_integer_categorical_feature(Cancer_dx_Age, "cancer_dx_age", train_ds)
Relative_Age_1_encoded = encode_integer_categorical_feature(Relative_Age_1, "rel_age_1", train_ds)
Relative_Age_2_encoded = encode_integer_categorical_feature(Relative_Age_2, "rel_age_2", train_ds)
Relative_Age_3_encoded = encode_integer_categorical_feature(Relative_Age_3, "rel_age_3", train_ds)

all_features = layers.concatenate(
    [
      Relative_Cancer_encoded, 
      Relative_Relation_encoded,
        Cancer_Dx_encoded, 
        Cancer_Dx_Type_encoded, 
        Ethnicity_1_encoded, 
        Ethnicity_2_encoded, 
        Known_BRCA_encoded, 
        Known_Cancer_encoded,
        Other_Cancer_encoded,
        Cancer_dx_Age_encoded,
        Relative_Age_1_encoded,
        Relative_Age_2_encoded,
        Relative_Age_3_encoded
    ]
)

# Multiple layers with dropout (for higher accuracy)
x = layers.Dense(187, activation="tanh", name = "Dense_1")(all_features)
x = layers.Dropout(0.2)(x)
x = layers.Dense(64, activation="relu", name = "Dense_2")(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(32, activation="relu", name = "Dense_3")(x)
output = layers.Dense(1, activation="sigmoid",name = "Outputlayer")(x)

model = keras.Model(all_inputs, output)

#Use Adam optimizer with custom learning rate

opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(opt, "binary_crossentropy", metrics=["accuracy"])

model.summary()

In [ ]:
keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

In [ ]:
!mkdir checkpoints

In [ ]:
checkpoint = ModelCheckpoint('./checkpoints/best_weights.tf', monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')

In [ ]:
history = model.fit(train_ds, epochs=70, validation_data=val_ds, callbacks=[checkpoint])

# Other

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())

In [ ]:
driver.get('https://en.wikipedia.org/wiki/List_of_contemporary_ethnic_groups')

In [ ]:
lstOfEthnicities = []
for i in range(1,497):
    #print(driver.find_element_by_xpath('/html/body/div[3]/div[3]/div[5]/div[1]/table[2]/tbody/tr['+str(i)+']/td[1]').text)
    lstOfEthnicities.append(driver.find_element_by_xpath('/html/body/div[3]/div[3]/div[5]/div[1]/table[2]/tbody/tr['+str(i)+']/td[1]').text)
lstOfEthnicities.append('caucasian')

In [ ]:
driver.quit()

In [ ]:
'norwegian' in lstOfEthnicities